In [6]:
from __future__ import print_function
from __future__ import division

import affine
from glob import glob
import os
import rasterio
from shutil import copyfile
import skimage.io as skio
import subprocess
import sys

sys.path.append('..')

from optical_vessel_detection.core import img_utils as iutils
import optical_vessel_detection.core.annotation_utils as autils

from optical_vessel_detection.support.info import fully_annotated, aois_by_date
from optical_vessel_detection.support import counting_tools
from optical_vessel_detection.support.regions import approximate_simplified_nk_eez, full_study_area
from optical_vessel_detection.support import notebook_utils

# Installing Existing Annotations

These instructions are specific to GFW's setup. You will need to adapt this
your specific situation. And, unfortunately, we cannot supply you with annotated
tiles.

### Creating tiles

* Make a directory called `data`
* Make a directory called `data/pair_trawl_aug_tiles_768`

* Download the annotated images
    - `gsutil -m cp -rn gs://machine-learning-production/planet_images/pair_trawler_annotations data/`
    - `gsutil -m cp -rn gs://machine-learning-production/planet_images/general_annotations data/`
    - `gsutil -m cp -rn gs://machine-learning-production/planet_images/pair_trawler_supp_annotations data/`
    
* Then run the code below

Note: I have sometimes seen these `gsutil -m cp` commands hang even though they are finished. If this
happens, kill the command and try rerunning without the `-m`.

In [2]:
annotation_source = [
                      ('pair_trawler_annotations', autils.PropMaps.pair_trawlers),
                      ('general_annotations', autils.PropMaps.negatives),
                      ('pair_trawler_supp_annotations', autils.PropMaps.pair_trawlers)
                    ]
count = 0
for src, prop_map in annotation_source: 
    count += len(glob("../data/{}/annotations/*.tif".format(src)))
print(count, "annotated images")

236 annotated images


In [ ]:
def build_augmented_tiles():
    for src, prop_map in annotation_source: 
        for ann_img_path in glob("../data/{}/annotations/*.tif".format(src)):
            name = os.path.splitext(os.path.basename(ann_img_path))[0]
            [raw_img_path] = glob("../data/{}/raw/{}*.tif".format(src, name))
            print()
            print(name)
            autils.create_tiles_from_annotated_img(raw_img_path, ann_img_path, 768, 
                                                  '../data/pair_trawl_aug_tiles_768/',
                                                  prop_map)
        print()
        
build_augmented_tiles()


20180826_012510_103d_3B_Visual
0000000000
20180524_013112_104b_3B_Visual
1000000000000000000000000000000000000000000000
20180821_011406_0f46_3B_Visual
11000000
20180522_012941_0f49_3B_Visual
1100000000000000000
20180826_011534_0f3b_3B_Visual
0
20180621_012539_1049_3B_Visual
11111111111100000000000000000000000000000
20180905_013636_1002_3B_Visual
000000000000000000000
20180818_012343_1033_3B_Visual
0000A
20171024_012047_0f35_3B_Visual
00
20180523_013323_1044_3B_Visual
1110000000000000000000000
20180821_011629_0f4d_3B_Visual
000000000000000
20180522_012622_1002_3B_Visual
111111111111111111111111100000000000000000000000000000000000000000000000000000000
20180814_013625_1010_3B_Visual
0000000000000000000000000000
20180811_011336_1053_3B_Visual
000000
20180607_013625_1051_3B_Visual
11111111111111111111111100000000000000000000000000000
20180819_012759_101f_3B_Visual
000000000
20180820_012307_0f32_3B_Visual
000000000000
20180713_012825_1012_3B_Visual
111111111000000000000000
20180814_012646_1

In [3]:
for src, prop_map in [('pair_trawler_annotations', autils.PropMaps.pair_trawlers),
                      ('general_annotations', autils.PropMaps.negatives),
                      ('pair_trawler_supp_annotations', autils.PropMaps.pair_trawlers)]: 
    print(src, len(glob("../data/{}/annotations/*.tif".format(src))))
    

pair_trawler_annotations 163
general_annotations 39
pair_trawler_supp_annotations 34


## Uploading tiles

This example is specific to the setup at GFWs setup, you wil need to adapt it to your own
directory structure.

* Upload the tiles to GCS (don't inadvertently clobber existing tiles)
    - `cd ~/Documents/GlobalFishingWatch/Code/image-vessel-identification/data/pair_trawl_suptiles_768/`
    - `gsutil -m cp -rn *.png gs:/machine-learning-dev-ttl-120d/image_id/pair_trawl_aug_tiles_768/`
* Update `gcs_tile_location` in `pair_trawl_tile_data.py`

## Commands

### Training

    python  deploy_cloudml.py \
                     --model_name pair_trawler_model_v1_1
                     --job_name v1_1_aug 
                     --config_file deploy_4p100.yaml \
                     --split -1 
    
### Inference

       python -m source.process_nk_images \
                     --source gs://machine-learning-production/planet_images/nk_detection/ \
                     --target gs://machine-learning-production/planet_paper/gpu_image_detections_1_1_aug \
                     --detections-table machine_learning_dev_ttl_120d.detected_pair_trawlers_1_1_aug \
                     --date 2018-09-29

# Creating New Annotations

1. Create a "raw" file scene using the code below. This is a geotif
   rotated to remove most white space. The transforms information is
   kept correct so that we can still map the annotations to lat, lon.
2. Place copies of this "raw" file in the `raw` and `annotations` directores.
3. Annotate the image using any drawing program. Trawler pairs should be indicated
   by lines between the trawlers. Other vessels should be indicated with lines
   from bow to stern. False positives hould be indicated by lines or dots centered
   on the false positive. The colors to use are:
   
     - Pair trawlers: red (255, 0, 0)
     - Single trawlers: pink (255, 128, 128)
     - Other vessels: orange (255, 128, 0)
     - False positive: blue (0, 0, 255)
     
   There are other other annotations for hot pixel artifacts and encounters that were used
   in earlier annotation schemes, but they aren't necessary at this point. We aren't
   classifying encounters and hot pixels can be marked as normal false positives.
   
   *N.B. anitaliasing will shift colors and break the annotation. If using a drawing
    program that uses antialiasing (e.g., Mac Preview) use thicker lines to work around
    this.*

In [3]:
def add_scenes_to_anotate(names, base_src_path, base_tgt_path):
    for x in names:
        src_path = os.path.join([base_src_path, x])
        tgt_path = os.path.join([base_tgt_path, x])
        iutils.create_straightened(src_path, tgt_path)
        
add_scenes_to_anotate([], "BASE_SRC_PATH", "BASE_TGT_PATH")